# チャットボットの作成

## 1.準備

In [ ]:
# 必要なモジュールをインポート
from dotenv import load_dotenv
from typing import TypedDict, Annotated, Any
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from IPython.display import Image, display
import uuid # UUID（ランダムなID）生成用
from langgraph.checkpoint.memory import InMemorySaver # メモリ保存用

# 環境変数の読み込み
load_dotenv()

# モデル名
MODEL_NAME = "gpt-5-mini"

## 2. ステートとノードの定義

In [ ]:
# 1. ステートの定義
class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

# 2. プロンプトとモデルの準備
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは親切なアシスタントです。"),
    MessagesPlaceholder(variable_name="messages")
])

model = ChatOpenAI(model=MODEL_NAME)

# チェーンの構築
my_chain = prompt | model

# 3. ノードの定義
def chatbot_node(state: State) -> dict[str, Any]:
    response = my_chain.invoke(state)
    
    # 更新差分（AIのメッセージ）を返します
    return {"messages": [response]}

## 3. グラフの作成

In [ ]:
# 4. グラフの構築
builder = StateGraph(State)

builder.add_node("chatbot", chatbot_node)

# エッジの定義 (START -> chatbot -> END)
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

# Checkpointerの初期化
memory = InMemorySaver()

# コンパイル時にcheckpointerを渡す
# これにより、グラフはスレッドIDに基づいて状態を保存・復元できるようになる
agent = builder.compile(checkpointer=memory)

# グラフの可視化
display(Image(agent.get_graph().draw_mermaid_png()))

## 4.メインループ

In [ ]:
# ランダムなUUIDを生成して thread_id に設定
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

while True:
    # ユーザーからの質問を受付
    user_input = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if user_input.strip() == "":
        break
    print(f"質問：{user_input}")
    
    # エージェントを実行し、応答をストリーミング表示
    # configを渡すことで、メモリから過去の会話をロードします
    for chunk, metadata in agent.stream(
        {"messages": [{"role": "user", "content": user_input}]},
        config=config, 
        stream_mode="messages"
    ):
        # chunkは AIMessageChunk オブジェクトなどが返ってきます
        if hasattr(chunk, "content") and chunk.content:
            print(chunk.content, end="", flush=True)
            
    print() # 改行

print("\n---ご利用ありがとうございました！---")